## 1. Setup & Installation

In [ ]:
# Clone the repo (only needed in Colab)
import os
if not os.path.exists('DaZZLeD'):
    !git clone https://github.com/D13ya/DaZZLeD.git
    %cd DaZZLeD/ml-core
else:
    %cd DaZZLeD/ml-core

!pip install -q -r requirements.txt

In [ ]:
import sys
import torch
import torch.nn.functional as F
from pathlib import Path

# Add project root to path
sys.path.insert(0, str(Path.cwd()))

from models.recursive_student import RecursiveHasher

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Model Architecture Test

In [ ]:
# Initialize model with default params
STATE_DIM = 128
HASH_DIM = 96
RECURSION_STEPS = 16

model = RecursiveHasher(state_dim=STATE_DIM, hash_dim=HASH_DIM)
model.eval()

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Model size: ~{total_params * 4 / 1024 / 1024:.2f} MB (float32)")

In [ ]:
# Test forward pass with dummy input
batch_size = 4
image_size = 224

dummy_img = torch.randn(batch_size, 3, image_size, image_size)
dummy_state = torch.zeros(batch_size, STATE_DIM)

with torch.no_grad():
    next_state, hash_out = model(dummy_img, dummy_state)

print(f"Input image shape: {dummy_img.shape}")
print(f"Input state shape: {dummy_state.shape}")
print(f"Output state shape: {next_state.shape}")
print(f"Output hash shape: {hash_out.shape}")
print(f"Hash L2 norm (should be ~1.0): {torch.norm(hash_out, dim=1).mean():.4f}")

## 3. Recursive Inference Test

The key innovation is running the model recursively 16 times, refining the hash at each step.

In [ ]:
def recursive_inference(model, image, steps=16):
    """Run recursive inference for the specified number of steps."""
    batch_size = image.size(0)
    state = torch.zeros(batch_size, STATE_DIM, device=image.device)
    
    hashes = []
    with torch.no_grad():
        for step in range(steps):
            state, hash_out = model(image, state)
            hashes.append(hash_out.clone())
    
    return hashes

# Run recursive inference
hashes = recursive_inference(model, dummy_img, steps=RECURSION_STEPS)

print(f"Generated {len(hashes)} hash vectors")
print(f"Final hash shape: {hashes[-1].shape}")

In [ ]:
# Analyze hash stability across recursive steps
import matplotlib.pyplot as plt

# Compute cosine similarity between consecutive steps
similarities = []
for i in range(1, len(hashes)):
    sim = F.cosine_similarity(hashes[i], hashes[i-1], dim=1).mean().item()
    similarities.append(sim)

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(range(1, len(hashes)), similarities, 'b-o')
plt.xlabel('Recursion Step')
plt.ylabel('Cosine Similarity with Previous')
plt.title('Hash Convergence Over Recursion')
plt.ylim(0, 1.1)
plt.grid(True)

# Compute similarity to final hash
final_hash = hashes[-1]
similarities_to_final = []
for h in hashes:
    sim = F.cosine_similarity(h, final_hash, dim=1).mean().item()
    similarities_to_final.append(sim)

plt.subplot(1, 2, 2)
plt.plot(range(len(hashes)), similarities_to_final, 'r-o')
plt.xlabel('Recursion Step')
plt.ylabel('Cosine Similarity to Final Hash')
plt.title('Convergence to Final Hash')
plt.ylim(0, 1.1)
plt.grid(True)

plt.tight_layout()
plt.show()

## 4. Adversarial Robustness Test

Test if small perturbations to input cause large changes in hash (they shouldn't after recursive refinement).

In [ ]:
def test_perturbation_robustness(model, image, epsilon_range, steps=16):
    """Test hash stability under input perturbations."""
    results = []
    
    # Get baseline hash
    baseline_hashes = recursive_inference(model, image, steps)
    baseline_final = baseline_hashes[-1]
    
    for epsilon in epsilon_range:
        # Add random noise
        noise = torch.randn_like(image) * epsilon
        perturbed = image + noise
        
        # Get perturbed hash
        perturbed_hashes = recursive_inference(model, perturbed, steps)
        perturbed_final = perturbed_hashes[-1]
        
        # Compute similarity
        sim = F.cosine_similarity(baseline_final, perturbed_final, dim=1).mean().item()
        results.append((epsilon, sim))
        print(f"Epsilon={epsilon:.4f}: Cosine Similarity={sim:.4f}")
    
    return results

# Test with various noise levels
epsilons = [0.001, 0.01, 0.05, 0.1, 0.2, 0.5]
test_img = torch.randn(1, 3, 224, 224)
results = test_perturbation_robustness(model, test_img, epsilons)

In [ ]:
# Plot robustness results
epsilons, sims = zip(*results)

plt.figure(figsize=(8, 5))
plt.plot(epsilons, sims, 'g-o', linewidth=2, markersize=8)
plt.axhline(y=0.9, color='r', linestyle='--', label='0.9 threshold')
plt.xlabel('Noise Level (epsilon)', fontsize=12)
plt.ylabel('Cosine Similarity to Original', fontsize=12)
plt.title('Hash Robustness to Input Perturbations', fontsize=14)
plt.xscale('log')
plt.ylim(0, 1.1)
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

## 5. ONNX Export Test

In [ ]:
import onnx
import onnxruntime as ort
import numpy as np

# Export to ONNX
onnx_path = "test_model.onnx"

model.eval()
dummy_img = torch.randn(1, 3, 224, 224)
dummy_state = torch.zeros(1, STATE_DIM)

torch.onnx.export(
    model,
    (dummy_img, dummy_state),
    onnx_path,
    input_names=["image", "prev_state"],
    output_names=["next_state", "hash"],
    opset_version=14,
    dynamic_axes={
        "image": {0: "batch"},
        "prev_state": {0: "batch"},
        "next_state": {0: "batch"},
        "hash": {0: "batch"},
    },
)

print(f"Exported ONNX model to {onnx_path}")
print(f"File size: {os.path.getsize(onnx_path) / 1024:.2f} KB")

In [ ]:
# Validate ONNX model
onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)
print("ONNX model validation passed!")

# Test ONNX runtime inference
session = ort.InferenceSession(onnx_path)

# Run inference
test_img = np.random.randn(1, 3, 224, 224).astype(np.float32)
test_state = np.zeros((1, STATE_DIM), dtype=np.float32)

outputs = session.run(None, {"image": test_img, "prev_state": test_state})
next_state_onnx, hash_onnx = outputs

print(f"ONNX output state shape: {next_state_onnx.shape}")
print(f"ONNX output hash shape: {hash_onnx.shape}")
print(f"ONNX hash L2 norm: {np.linalg.norm(hash_onnx, axis=1).mean():.4f}")

In [ ]:
# Compare PyTorch vs ONNX outputs
with torch.no_grad():
    pt_state, pt_hash = model(torch.from_numpy(test_img), torch.from_numpy(test_state))

pt_hash_np = pt_hash.numpy()
pt_state_np = pt_state.numpy()

hash_diff = np.abs(pt_hash_np - hash_onnx).max()
state_diff = np.abs(pt_state_np - next_state_onnx).max()

print(f"Max hash difference (PyTorch vs ONNX): {hash_diff:.8f}")
print(f"Max state difference (PyTorch vs ONNX): {state_diff:.8f}")

if hash_diff < 1e-5 and state_diff < 1e-5:
    print("✅ ONNX export matches PyTorch output!")
else:
    print("⚠️ Warning: Numerical differences detected")

## 6. Latency Benchmark

In [ ]:
import time

def benchmark_inference(model, device, num_runs=100, warmup=10):
    """Benchmark recursive inference latency."""
    model = model.to(device)
    model.eval()
    
    test_img = torch.randn(1, 3, 224, 224).to(device)
    
    # Warmup
    for _ in range(warmup):
        _ = recursive_inference(model, test_img, steps=16)
    
    if device.type == 'cuda':
        torch.cuda.synchronize()
    
    # Benchmark
    times = []
    for _ in range(num_runs):
        if device.type == 'cuda':
            torch.cuda.synchronize()
        start = time.perf_counter()
        
        _ = recursive_inference(model, test_img, steps=16)
        
        if device.type == 'cuda':
            torch.cuda.synchronize()
        times.append(time.perf_counter() - start)
    
    return times

# Benchmark on CPU
cpu_times = benchmark_inference(model, torch.device('cpu'), num_runs=50)
print(f"CPU Latency: {np.mean(cpu_times)*1000:.2f} ± {np.std(cpu_times)*1000:.2f} ms")

# Benchmark on GPU if available
if torch.cuda.is_available():
    gpu_times = benchmark_inference(model, torch.device('cuda'), num_runs=100)
    print(f"GPU Latency: {np.mean(gpu_times)*1000:.2f} ± {np.std(gpu_times)*1000:.2f} ms")

## 7. Summary

This notebook validates:
1. ✅ Model architecture and parameter count
2. ✅ Forward pass correctness
3. ✅ Recursive inference convergence
4. ✅ Perturbation robustness
5. ✅ ONNX export and validation
6. ✅ Inference latency

**Next steps:**
- Train the model using `training/train.py` (see COLAB.md)
- Export trained model to ONNX
- Integrate ONNX model into Go runtime